In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
listed = drive.ListFile({'q': "title contains '.pkl' and 'root' in parents"}).GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))


title 5s.pkl, id 1mt5-OWZPJkcSJDI84MnIluB6iIEEoosL
title 5s.pkl, id 18mwG2e4SJHZAOD2oyaEv0-7_beZe-QcH
title refined1.pkl, id 1fo-gIx5b8J9eiZcMFik9zK7OTCgcIJnT
title base_df.pkl, id 1iYfS2oBhz4l5YduHbXvwlgU4O-NAZDB8


In [0]:
#Needed to import left-right dataframe
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '1fo-gIx5b8J9eiZcMFik9zK7OTCgcIJnT'

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
f=pickle.load(downloaded)

In [5]:
f.head(8)

,__label__,tweets
0,__label__Right,problema risolve rinviando primavera vorrebbe ...
1,__label__Left,_lica _ _xy _piu_smocci _g _lica _ _xy _piu_sm...
2,__label__Unknown,popolo manca laltro pi | _ racconto inedito sa...
3,__label__Right,gioved\xc3\xac 5 dicembre repubblica impuniti ...
4,__label__Left,gran bretagna paese pi ricco industrializzato ...
5,__label__Unknown,ambiente costruito piattaforme digitali va ver...
6,__label__Right,parole parassiti complimenti capitano molto be...
7,__label__Left,genova + sardine + de andr meraviglia assoluta...


In [0]:
import pandas as pd
import numpy as np
df=f




In [7]:
df=f
df=df[df.loc[:,'__label__']!='__label__Unknown']
df.head()

,__label__,tweets
0,__label__Right,problema risolve rinviando primavera vorrebbe ...
1,__label__Left,_lica _ _xy _piu_smocci _g _lica _ _xy _piu_sm...
3,__label__Right,gioved\xc3\xac 5 dicembre repubblica impuniti ...
4,__label__Left,gran bretagna paese pi ricco industrializzato ...
6,__label__Right,parole parassiti complimenti capitano molto be...


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

training_data, testing_data = train_test_split(df,random_state = 2000)
Y_train=training_data['__label__']
Y_test=testing_data['__label__']
X_train=training_data['tweets']
X_test=testing_data['tweets']
field='tweets'
field

'tweets'

In [9]:
print(X_test)

3034    leggo difende contante adduce ragione \xe2\x80...
162     potrei accenderti riscaldamento forno fusorio ...
2691    _piu_smocci quando dx sx battista rotta coglio...
3258    voce persone raccontare natura emergere grandi...
376     crolla parte viadotto sullautostrada altezza m...
                              ...                        
2310    apoteosi nonsense vorrebbero impedirti vedere ...
1219    stato incubo marito grasso crescano pure poi e...
1153    piombino conferisce cittadinanza onoraria lili...
3172    aggirare divieti fascisti spostati social netw...
757     _imerub confermo sottoscrivo giustamente qualc...
Name: tweets, Length: 550, dtype: object


In [0]:
def extract_features(df,field,training_data,testing_data,type="binary"):
  if "binary" in type:
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set, test_feature_set, cv
   
  elif "counts" in type:
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
  else:    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer
  
    
   

In [0]:
 X_train,X_test,feature_transformer = extract_features(df, field,training_data,testing_data, type="counts")
    

In [0]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

In [22]:
model=scikit_log_reg.fit(X_train,Y_train)

[LibLinear]

In [23]:
model.score(X_test,Y_test)
#Note that the model score is obtained training the algorithm with 'counts' method
#if we use binary we obtain a 84%accuracy 
#If we use TfIdf we obtain a 79%accuracy


0.7945454545454546

In [0]:
#Needed to import m5s dataframe
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '1mt5-OWZPJkcSJDI84MnIluB6iIEEoosL'

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
cinques=pickle.load(downloaded)

In [25]:
cinques.head()

,_label,tweets
0,label_5s,chiesto documento aprire profilo potuto tormen...
1,label_5s,grillino confessa cognome beh dipende verament...
2,label_5s,malties people forced prime minister resign vi...
3,label_5s,vendicare perch determinato crisi prosit baron...
4,label_5s,spero vi uoso amministrazione rigenerazione ma...


In [26]:
#Set the tweets of dataframe as array to test the 5s profiles
X5_test=cinques['tweets']
print(X5_test)

0      chiesto documento aprire profilo potuto tormen...
1      grillino confessa cognome beh dipende verament...
2      malties people forced prime minister resign vi...
3      vendicare perch determinato crisi prosit baron...
4      spero vi uoso amministrazione rigenerazione ma...
                             ...                        
198    disse magistratura cancro attacco pi subdolo p...
199    @ circomassimo sensate fatti bonafede battagli...
200    reddito cittadinanza libera erogazione stranie...
201    gaml action depends actions intentions speakin...
202    piove dirotto scricchiolio pochi simil volpino...
Name: tweets, Length: 203, dtype: object


In [0]:
#Let's transform the Tweets with the funciton that makes it suitable for do predictions
#Don't forget that the model is trained on left and right profile
#Using 'counts' 

X5_test,X5_train,feature_transformer = extract_features(cinques, field,training_data,testing_data, type="counts")


In [30]:
tested=model.predict(X5_test)
print(tested)


['__label__Left' '__label__Left' '__label__Left' ... '__label__Right'
 '__label__Right' '__label__Right']


1650

In [0]:
sx=0
dx=0
for i in np.arange(len(tested)):
  if tested[i]=='__label__Left' : sx=sx+1
  else : dx=dx+1

In [0]:
print(sx,dx)

831 819


In [0]:
#Let's transform the Tweets with the funciton that makes it suitable for do predictions
#Don't forget that the model is trained on left and right profile
#Using 'binary' method 
X5_test,X5_train,feature_transformer = extract_features(cinques, field,training_data,testing_data, type="binary")


In [0]:
tested=model.predict(X5_test)
print(tested)

['__label__Left' '__label__Left' '__label__Left' ... '__label__Right'
 '__label__Right' '__label__Left']


In [0]:
sx=0
dx=0
for i in np.arange(len(tested)):
  if tested[i]=='__label__Left' : sx=sx+1
  else : dx=dx+1

In [0]:
print(sx,dx)

1055 595


In [0]:
#Let's transform the Tweets with the funciton that makes it suitable for do predictions
#Don't forget that the model is trained on left and right profile
#Using 'counts' method of tfidf
X5_test,X5_train,feature_transformer = extract_features(cinques, field,training_data,testing_data, type='no')


In [0]:
tested=model.predict(X5_test)
print(tested)

['__label__Right' '__label__Right' '__label__Right' ... '__label__Right'
 '__label__Right' '__label__Right']


In [0]:
sx=0
dx=0
for i in np.arange(len(tested)):
  if tested[i]=='__label__Left' : sx=sx+1
  else : dx=dx+1

In [0]:
print(sx,dx)

0 1650
